<a href="https://colab.research.google.com/github/CRosero/aml-project/blob/master/train_step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the dataset from drive.
( You can find the zipped folder [here](https://drive.google.com/file/d/1XsRmyQYHfgRFJCOueXpJ37yyOCrKHO-W/view?usp=sharing))

In [40]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/gdrive/')
data_path = "/content/data"

use_complete_dataset = True

if (use_complete_dataset == True) and (not os.path.isfile('/content/data.zip')):
  print("download entire dataset")
  !gdown --id 1A2dBwPlCyXHTqmG1LRvPfVm6K21jWwUI # 3-5 min
  !jar xf  "/content/data.zip"
elif (use_complete_dataset == False) and (not os.path.isfile('/content/data.zip')):
  # Load cropped dataset containing only 10 images
  print("download cropped dataset")
  !gdown --id 1gPcwDJsNpyqcjKu225hnIxkURpjUA08i   
  !jar xf  "/content/data.zip"
else:
  print("dataset already downloaded")

if not os.path.isdir('/content/data'):
  print("Dataset doesn't exist")

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
dataset already downloaded


# Cloning the repository from github

In [41]:
# Clone the entire repo.
repo_path = "/content/cloned-repo"
if not os.path.isdir(repo_path):
  !git clone -l -s https://github.com/CRosero/aml-project.git cloned-repo
  %cd cloned-repo
else:
  print("Repository already cloned")
%cd /content/cloned-repo
!ls

Repository already cloned
/content/cloned-repo
dataset     image_output  __pycache__  train_step_2-experiment.ipynb  utils.py
eval.ipynb  loss.py	  README.md    train_step_2.ipynb
eval.py     model	  train.py     train_step_3.ipynb


In [42]:

!git pull

Already up to date.


# Importing the libraries

In [43]:
import argparse
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from model.build_BiSeNet import BiSeNet
import torch
from torch.utils.tensorboard import SummaryWriter
from loss import CrossEntropy2d
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from loss import DiceLoss
import torch.cuda.amp as amp
import os
import os.path as osp
import random
import matplotlib.pyplot as plt
import collections
import torchvision
from torchvision.transforms import InterpolationMode
from torch.utils import data
from PIL import Image
import torch.nn.functional as F

import json
# Dataset class:
from dataset.cityscapesDataSet import cityscapesDataSet
from dataset.GTA5DataSet import GTA5DataSet
# Discriminator
from model.discriminator import FCDiscriminator

In [44]:
def enable_cuda(obj, gpu):
  if torch.cuda.is_available():
    return obj.cuda(gpu)
  else:
    return obj

In [45]:
# Load TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
def loss_calc(pred, labels, gpu):
    """
    This function returns cross entropy loss for semantic segmentation
    """
    # out shape batch_size x channels x h x w -> batch_size x channels x h x w
    # label shape h x w x 1 x batch_size  -> batch_size x 1 x h x w
    labels = Variable(labels.long()).cuda(gpu)
    labels = enable_cuda(labels, gpu)
    criterion = CrossEntropy2d()
    criterion = enable_cuda(criterion, gpu)

    return criterion(pred, labels)


def lr_poly(base_lr, iter, max_iter, power):
    return base_lr * ((1 - float(iter) / max_iter) ** (power))


def adjust_learning_rate(optimizer, i_iter):
    lr = lr_poly(args.learning_rate, i_iter, args.num_steps, args.power)
    optimizer.param_groups[0]['lr'] = lr
    if len(optimizer.param_groups) > 1:
        optimizer.param_groups[1]['lr'] = lr * 10


def adjust_learning_rate_D(optimizer, i_iter):
    lr = lr_poly(args.learning_rate_D, i_iter, args.num_steps, args.power)
    optimizer.param_groups[0]['lr'] = lr
    if len(optimizer.param_groups) > 1:
        optimizer.param_groups[1]['lr'] = lr * 10


In [47]:
IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)

MODEL = 'BiseNet'
BATCH_SIZE = 2
ITER_SIZE = 1
NUM_WORKERS = 4
DATA_DIRECTORY = '/content/data/'
DATA_LIST_PATH = './train.txt'
IGNORE_LABEL = 255
INPUT_SIZE = '600,304'#'1280,720'
DATA_DIRECTORY_TARGET = './data/Cityscapes'
DATA_LIST_PATH_TARGET = './train.txt'
INPUT_SIZE_TARGET = '600,304'#'1024,512'
LEARNING_RATE = 2.5e-2
MOMENTUM = 0.9
NUM_CLASSES = 19
NUM_STEPS = 500//BATCH_SIZE
NUM_STEPS_STOP = 150
POWER = 0.9
RANDOM_SEED = 1234
RESTORE_FROM =  '/gdrive/MyDrive/Project_AML/Models/checkpoints_segNet/'
SAVE_NUM_IMAGES = 2
SAVE_PRED_EVERY = 10
SNAPSHOT_DIR = '/gdrive/MyDrive/Project_AML/Models/snapshots_segNet/'
WEIGHT_DECAY = 1e-4

LEARNING_RATE_D = 1e-4
LAMBDA_SEG = 0.1
LAMBDA_ADV_TARGET = 0.0002
GAN = 'Vanilla'

TARGET = 'Cityscapes'
SET = 'train'

In [48]:
def get_arguments(params=[]):
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """
        
    # basic parameters
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, default=MODEL,
                        help="available options : BiseNet")
    parser.add_argument("--target", type=str, default=TARGET,
                        help="available options : Cityscapes")
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE,
                        help="Number of images sent to the network in one step.")
    parser.add_argument("--iter-size", type=int, default=ITER_SIZE,
                        help="Accumulate gradients for ITER_SIZE iterations.")
    parser.add_argument("--num-workers", type=int, default=NUM_WORKERS,
                        help="number of workers for multithread dataloading.")
    parser.add_argument("--data-dir", type=str, default=DATA_DIRECTORY,
                        help="Path to the directory containing the source dataset.")
    parser.add_argument("--data-list", type=str, default=DATA_LIST_PATH,
                        help="Path to the file listing the images in the source dataset.")
    parser.add_argument("--ignore-label", type=int, default=IGNORE_LABEL,
                        help="The index of the label to ignore during the training.")
    parser.add_argument("--input-size", type=str, default=INPUT_SIZE,
                        help="Comma-separated string with height and width of source images.")
    parser.add_argument("--data-dir-target", type=str, default=DATA_DIRECTORY_TARGET,
                        help="Path to the directory containing the target dataset.")
    parser.add_argument("--data-list-target", type=str, default=DATA_LIST_PATH_TARGET,
                        help="Path to the file listing the images in the target dataset.")
    parser.add_argument("--input-size-target", type=str, default=INPUT_SIZE_TARGET,
                        help="Comma-separated string with height and width of target images.")
    parser.add_argument("--is-training", action="store_true",
                        help="Whether to updates the running means and variances during the training.")
    parser.add_argument("--learning-rate", type=float, default=LEARNING_RATE,
                        help="Base learning rate for training with polynomial decay.")
    parser.add_argument("--learning-rate-D", type=float, default=LEARNING_RATE_D,
                        help="Base learning rate for discriminator.")
    parser.add_argument("--lambda-seg", type=float, default=LAMBDA_SEG,
                        help="lambda_seg.")
    parser.add_argument("--lambda-adv-target", type=float, default=LAMBDA_ADV_TARGET,
                        help="lambda_adv for adversarial training.")
    parser.add_argument("--momentum", type=float, default=MOMENTUM,
                        help="Momentum component of the optimiser.")
    parser.add_argument("--not-restore-last", action="store_true",
                        help="Whether to not restore last (FC) layers.")
    parser.add_argument("--num-classes", type=int, default=NUM_CLASSES,
                        help="Number of classes to predict (including background).")
    parser.add_argument("--num-steps", type=int, default=NUM_STEPS,
                        help="Number of training steps.")
    parser.add_argument("--num-steps-stop", type=int, default=NUM_STEPS_STOP,
                        help="Number of training steps for early stopping.")
    parser.add_argument("--power", type=float, default=POWER,
                        help="Decay parameter to compute the learning rate.")
    parser.add_argument("--random-mirror", action="store_true",
                        help="Whether to randomly mirror the inputs during the training.")
    parser.add_argument("--random-scale", action="store_true",
                        help="Whether to randomly scale the inputs during the training.")
    parser.add_argument("--random-seed", type=int, default=RANDOM_SEED,
                        help="Random seed to have reproducible results.")
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM,
                        help="Where restore model parameters from.")
    parser.add_argument("--save-num-images", type=int, default=SAVE_NUM_IMAGES,
                        help="How many images to save.")
    parser.add_argument("--save-pred-every", type=int, default=SAVE_PRED_EVERY,
                        help="Save summaries and checkpoint every often.")
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR,
                        help="Where to save snapshots of the model.")
    parser.add_argument("--weight-decay", type=float, default=WEIGHT_DECAY,
                        help="Regularisation parameter for L2-loss.")
    parser.add_argument("--gpu", type=int, default=0,
                        help="choose gpu device.")
    parser.add_argument("--set", type=str, default=SET,
                        help="choose adaptation set.")
    parser.add_argument("--gan", type=str, default=GAN,
                        help="choose the GAN objective.")
    parser.add_argument('--context_path', type=str, default="resnet101",
                        help='The context path model you are using, resnet18, resnet101.')


    args = parser.parse_args(params)
    return args



In [49]:
#def main(params):
"""Create the model and start the training."""
params = []
args = get_arguments(params)

w, h = map(int, args.input_size.split(','))
input_size = (w, h)

w, h = map(int, args.input_size_target.split(','))
input_size_target = (w, h)

cudnn.benchmark = True
cudnn.enabled = True
gpu = args.gpu

# Create network
if args.model == 'BiseNet':
  model = BiSeNet(num_classes=args.num_classes, context_path= args.context_path)

# Set the network to train mode
model.train()
model = enable_cuda(model, args.gpu)

# init D
model_D = FCDiscriminator(num_classes=args.num_classes)

# Set the discriminators to train mode
model_D.train()
model_D = enable_cuda(model_D, args.gpu)

# check if the snapshot dir exists, if not create it
if not os.path.exists(args.snapshot_dir):
  os.makedirs(args.snapshot_dir)

        

In [50]:
# Path
source_data_root_path = os.path.join(args.data_dir, "GTA5") # /content/data/GTA5
target_data_root_path = os.path.join(args.data_dir, args.target) # /content/data/Cityscapes
source_train_path = os.path.join(source_data_root_path, "train.txt") # /content/data/GTA5/train.txt
target_root_path = os.path.join(target_data_root_path,  "train.txt")   # /content/data/Cityscapes/train.txt
info_path = os.path.join(source_data_root_path,  "info.json") # /content/data/GTA/info.json 

info_json = json.load(open(info_path))

# Datasets  
source_dataset = GTA5DataSet(source_data_root_path, source_train_path, info_json, crop_size=input_size, scale=args.random_scale, mirror=args.random_mirror, mean=IMG_MEAN)
target_dataset = cityscapesDataSet(target_data_root_path, target_root_path, info_json, crop_size=input_size_target, scale=False, mirror=args.random_mirror, mean=IMG_MEAN)

print("GTA: ", len(source_dataset))
print("Cityscapes: ", len(target_dataset))
img,label = source_dataset[0]
print ("GTA image", img.shape )
print ("GTA label", label.shape )
img, _ = target_dataset[0]
print ("Cityscapes image", img.shape )

# Create DataLoaders
trainloader = data.DataLoader(source_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)
trainloader_iter = enumerate(trainloader)

targetloader = data.DataLoader(target_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)
targetloader_iter = enumerate(targetloader)



GTA:  10
Cityscapes:  10
GTA image (3, 304, 600)
GTA label (304, 600)
Cityscapes image (3, 304, 600)


In [51]:
# Optimizer

# implement model.optim_parameters(args) to handle different models' lr setting
optimizer = optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.momentum, weight_decay=args.weight_decay)
optimizer.zero_grad()

optimizer_D = optim.Adam(model_D.parameters(), lr=args.learning_rate_D, betas=(0.9, 0.99))
optimizer_D.zero_grad()

if args.gan == 'Vanilla':
  bce_loss = torch.nn.BCEWithLogitsLoss()
elif args.gan == 'LS':
  bce_loss = torch.nn.MSELoss()

interp = torch.nn.Upsample(size=(input_size[1], input_size[0]), mode='bilinear')
interp_target = torch.nn.Upsample(size=(input_size_target[1], input_size_target[0]), mode='bilinear')

# labels for adversarial training
source_label_id = 0
target_label_id = 1



In [52]:
for i_iter in range(args.num_steps):

  loss_seg_value = 0
  loss_adv_target_value = 0
  loss_D_value = 0

  optimizer.zero_grad()
  adjust_learning_rate(optimizer, i_iter)

  optimizer_D.zero_grad()
  adjust_learning_rate_D(optimizer_D, i_iter)

  for sub_i in range(args.iter_size):

      # train G

      # don't accumulate grads in D
      for param in model_D.parameters():
          param.requires_grad = False

      # train with source images and labels
      _, batch = next(trainloader_iter)
      source_images, source_labels = batch
      source_images = Variable(source_images)
      source_images = enable_cuda(source_images, args.gpu)

      pred_source_result, pred_source_1, pred_source_2 = model(source_images)
      loss1 = loss_calc(pred_source_result, source_labels, args.gpu)
      loss2 = loss_calc(pred_source_1, source_labels, args.gpu)
      loss3 = loss_calc(pred_source_2, source_labels, args.gpu)
      loss_seg = loss1 + loss2 + loss3

      # proper normalization
      loss_seg = loss_seg / args.iter_size
      loss_seg.backward()
      print(loss_seg.data.cpu().numpy())
      print(loss_seg.data.cpu().numpy().shape)
      loss_seg_value += loss_seg.data.cpu().numpy() / args.iter_size

      # train with target images
      _, batch = next(targetloader_iter)
      target_images, _ = batch
      target_images = Variable(target_images).cuda(args.gpu)

      pred_target_result, pred_target_1, pred_target_2 = model(target_images)

      # generator vs. discriminator 
      D_out = model_D(F.softmax(pred_target_result))

      loss_adv_target = bce_loss(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label_id)).cuda(args.gpu))
      loss = args.lambda_adv_target * loss_adv_target
      loss = loss / args.iter_size
      loss.backward()
      loss_adv_target_value += loss_adv_target.data.cpu().numpy() / args.iter_size

      # train discriminator
      # bring back requires_grad
      for param in model_D.parameters():
          param.requires_grad = True

      # train with source
      pred_source_result = pred_source_result.detach()
      
      D_out_source = model_D(F.softmax(pred_source_result))
      loss_D_source = bce_loss(D_out_source, Variable(torch.FloatTensor(D_out_source.data.size()).fill_(source_label_id)).cuda(args.gpu))
      loss_D_source = loss_D_source / args.iter_size / 2
      loss_D_source.backward()

      loss_D_value += loss_D_source.data.cpu().numpy()

      # train with target
      pred_target_result = pred_target_result.detach()

      D_out_target = model_D(F.softmax(pred_target_result))

      loss_D_target = bce_loss(D_out_target, Variable(torch.FloatTensor(D_out_target.data.size()).fill_(target_label_id)).cuda(args.gpu))
      loss_D_target = loss_D_target / args.iter_size / 2
      loss_D_target.backward()

      loss_D_value += loss_D_target.data.cpu().numpy()

  optimizer.step()
  optimizer_D.step()

  print('exp = {}'.format(args.snapshot_dir))
  print('iter = {0:8d}/{1:8d}, loss_seg = {2:.3f}, loss_adv = {3:.3f}, loss_D = {4:.3f}'.format(i_iter, args.num_steps, loss_seg_value, loss_adv_target_value, loss_D_value))

  if i_iter >= args.num_steps_stop - 1:
      print ('save model ...')
      torch.save(model.state_dict(), osp.join(args.snapshot_dir, 'GTA5_' + str(args.num_steps_stop) + '.pth'))
      torch.save(model_D.state_dict(), osp.join(args.snapshot_dir, 'GTA5_' + str(args.num_steps_stop) + '_D.pth'))
      break

  if i_iter % args.save_pred_every == 0 and i_iter != 0:
      print ('taking snapshot ...')
      torch.save(model.state_dict(), osp.join(args.snapshot_dir, 'GTA5_' + str(i_iter) + '.pth'))
      torch.save(model_D.state_dict(), osp.join(args.snapshot_dir, 'GTA5_' + str(i_iter) + '_D.pth'))

10.274339
()
exp = /gdrive/MyDrive/Project_AML/Models/snapshots_segNet/
iter =        0/     250, loss_seg = 10.274, loss_adv = 0.690, loss_D = 0.693
9.11343
()
exp = /gdrive/MyDrive/Project_AML/Models/snapshots_segNet/
iter =        1/     250, loss_seg = 9.113, loss_adv = 0.693, loss_D = 0.693
8.176761
()
exp = /gdrive/MyDrive/Project_AML/Models/snapshots_segNet/
iter =        2/     250, loss_seg = 8.177, loss_adv = 0.696, loss_D = 0.693
7.028964
()
exp = /gdrive/MyDrive/Project_AML/Models/snapshots_segNet/
iter =        3/     250, loss_seg = 7.029, loss_adv = 0.698, loss_D = 0.692
7.4563913
()
exp = /gdrive/MyDrive/Project_AML/Models/snapshots_segNet/
iter =        4/     250, loss_seg = 7.456, loss_adv = 0.699, loss_D = 0.692


StopIteration: ignored

In [ ]:
! nvidia-smi

In [ ]:
%tensorboard --logdir=runs

In [ ]:
if __name__ == '__main__':
    params = [
        
    ]
    main(params)

In [ ]:
torch.cuda.memory_summary()
torch.cuda.empty_cache()

In [ ]:
  '''
  if args.restore_from[:4] == 'http' :
      saved_state_dict = model_zoo.load_url(args.restore_from)
  else:
      saved_state_dict = torch.load(args.restore_from)

  new_params = model.state_dict().copy()
  for i in saved_state_dict:
      # Scale.layer5.conv2d_list.3.weight
      i_parts = i.split('.')
      # print i_parts
      if not args.num_classes == 19 or not i_parts[1] == 'layer5':
          new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
          print(i_parts)
  model.load_state_dict(new_params)
  '''